#Student Name: Abhijit Sinha
#Student ID: 18195067

###### Week 3 Code taken from peer Eamon OGairbhith for this Etivity4

### Task:

###### Create a class that contains one tensor and provides the following methods:

1. Calculate the size of the tensor as a 2D tuple
2. Sum/subtract the tensor with another tensor of suitable size
3. Multiply the tensor with a tensor of suitable size
4. Calculate the determinant of the tensor, but only for tensors of size 2x2 (matrix)
5. Calculate the inverse of the tensor, but only for tensors of size 2x2
6. Calculate the cross product of the tensor with another tensor of suitable size

###### N.B.:
1. Use the Jupyter Notebook provided in the repository for week 4.
2. Limit your code to use for tensors of rank 1 and 2 (vectors and matrices)
3. Write your code without the use of imported modules.
4. Provide suitable exception handling to prevent the above operations from being used on unsuitable tensors.
5. Test your code with one example for each of the following scenarios:
    1. sum of a 4x4 with a 4x4 matrix
    2. multiplication of a 4x4 with a 4x4 matrix
    3. multiplication of a 4x4 matrix with a suitable vector
    4. determinant of a 2x2 matrix
    5. inverse of a 2x2 matrix
    6. cross-product of 2 suitable tensors

#### Proposed Solution:
1. Created a class Matrix and added existing functions as methods for matrix operations.
2. Implemented new functions for calculating:
    1. Matrix Determinant of NxN matrices
    2. Inverse of a 2x2 Matrix
    3. Cross product of a vector with another vector
    
3. Provided suitable excetion handling 
4. Tested the code with the following scenarios:
    1. sum of a 4x4 with a 4x4 matrix
    2. multiplication of a 4x4 with a 4x4 matrix
    3. multiplication of a 4x4 matrix with a suitable vector
    4. determinant of a 2x2 matrix
    5. inverse of a 2x2 matrix
    6. Proved that: A.A(inv) = Identity matrix
    7. cross-product of 2 suitable tensors

In [18]:
# Reset the variables in notebook
% reset

class Tensor:
    """ Creating a Tensor Class that instantiates a tensor (matrix, vector)"""

    def __init__(self, matrix):
        """Initialize a Tensor"""
        self.matrix = matrix
        self.shape = self.__size_matrix()

    def __str__(self):
        """Private method to print the result in string format"""
        return str(self.matrix)

    def get_matrix(self):
        """Return the matrix"""
        return self.matrix

    def __size_matrix(self):
        """Function to check size of a matrix"""
        return len(self.matrix), len(self.matrix[0])

    def __mat_init(self, size):
        """Function to create a empty matrix initialized with zeros"""

        num_row, num_col = size
        mat_null = [[0 for x in range(num_col)]
                       for y in range(num_row)]
        return tuple(mat_null)

    def matrix_dimensions_check(self, matrix_b):
        """Method to compare size of matrices for addition and subtraction"""
        if self.shape != matrix_b.shape:
            raise Exception("The matrices are of different dimensions")

    def add_sub_mat(self, b, fn_operator="add"):
        """ As per Abhi's advice, the original separate add
        and subtract functions have been combined in to one.
        """
        # Matrix Dimension Check function added in as per Abhi's advice
        self.matrix_dimensions_check(self)

        num_row, num_col = self.shape
        temp_value = self.__mat_init((num_row, num_col))
        for r in range(num_row):
            for c in range(num_col):
                if fn_operator == 'sub':
                    temp_value[r][c] = self.matrix[r][c] - b.matrix[r][c]
                else:
                    temp_value[r][c] = self.matrix[r][c] + b.matrix[r][c]
        return Tensor(tuple(temp_value))

    def multiply_matrices(self, matrix_b):
        """Function to perform multiplication. This methodology was taken from Vipul's
        solution. His method was a lot better than mine in both simplicity and functionality
        """
        num_row_a, num_col_a = self.shape
        num_row_b, num_col_b = matrix_b.shape

        if num_col_a != num_row_b:
            raise ValueError("The matrices in question cannot be multiplied")

        result = []
        # iterate on row of the first matrix
        for i, mat_list in enumerate(self.get_matrix()):
            internal_result = []
            for j in range(len(matrix_b.get_matrix()[i])):
                # extracts columns from 2nd Matrix for multiplication
                multiplier = list(zip(*matrix_b.get_matrix()))[j]
                internal_result.append((sum([x * y for x, y in zip(mat_list, multiplier)])))
            result.append(tuple(internal_result))
        return Tensor(tuple(result))

    def matrix_determinant(self):
        """Method to calculate determinant of 2x2 tensor"""
        if self.shape != (2, 2):
            raise ValueError("Only 2x2 tensor is supported")

        # Calculate the determinant
        determinant = (self.matrix[0][0] * self.matrix[1][1]) - \
                      (self.matrix[1][0] * self.matrix[0][1])
        return determinant

    def matrix_inverse(self):
        """Method to calculate the inverse of a 2x2 tensor"""
        if self.shape != (2, 2):
            raise ValueError("Only 2x2 tensor is supported")

        determinant = self.matrix_determinant()
        # Raise an exception if the determinant is 0 to avoid divide by zero
        if determinant == 0:
            raise Exception("Determinant is Zero")

        det_inv = 1 / determinant
        mat_00 = self.matrix[0][0] * det_inv
        mat_01 = self.matrix[0][1] * det_inv
        mat_10 = self.matrix[1][0] * det_inv
        mat_11 = self.matrix[1][1] * det_inv
        return Tensor(((mat_11, -mat_01), (-mat_10, mat_00)))

    def matrix_cross_product(self, matrix_b):
        """Method to calculate the cross product of two tensors of suitable size
        Implementation is for rank 3 vectors using the Right Hand Rule
        (a_x, a_y, a_z) x (b_x, b_y, b_z) = (a_y.b_z - a_z.b_y, a_z.b_x - a_x.b_z, a_x.b_y - a_y.b_x)
        """
        num_row_a, num_col_a = self.shape
        num_row_b, num_col_b = matrix_b.shape

        if num_row_a != 3 or num_row_b != 3 or num_col_a != 1 or num_col_b != 1:
            raise ValueError("Incompatible tensor for cross product")

        # Apply the cross product rule
        a = self.get_matrix()
        b = matrix_b.get_matrix()
        cp_tensor = (((a[1][0] * b[2][0]) - (a[2][0] * b[1][0])),
                     ((a[2][0] * b[0][0]) - (a[0][0] * b[2][0])),
                     ((a[0][0] * b[1][0]) - (a[1][0] * b[0][0])))

        return cp_tensor


"""Example of 2 x 2 Matrices For Testing"""
a_2x2 = ((1, 2), (5, 6))
b_2x2 = ((-1, 3), (-2, 6))

"""Example of 4 x 4 Matrices For Testing provided by Abhi"""
a_4x4 = ((2, 1, 5, 7), (6, -2, 0, 3), (3, 2, -4, 1), (9, 7, 2, 3))
b_4x4 = ((1, 0, 2, 1), (-2, 1, 5, 2), (3, -2, 6, 4), (2, 0, 1, -2))

"""4x1 Column vectors"""
a_4x1 = ((1,), (-3,), (5,), (8,))
b_4x1 = ((2,), (-1,), (-2,), (10,))

"""3x1 Column vectors"""
a_3x1 = ((1,), (2,), (3,))
b_3x1 = ((4,), (5,), (6,))

A_2X2 = Tensor(a_2x2)
B_2x2 = Tensor(b_2x2)
A_4x4 = Tensor(a_4x4)
B_4x4 = Tensor(b_4x4)
A_4x1 = Tensor(a_4x1)
B_4x1 = Tensor(b_4x1)
A_3x1 = Tensor(a_3x1)
B_3x1 = Tensor(b_3x1)

# Test methods
print("A_2X2 matrix     :{}".format(A_2X2.get_matrix()))
print("B_2x2 matrix     :{}".format(B_2x2.get_matrix()))
print("A_4x4 matrix     :{}".format(A_4x4.get_matrix()))
print("B_4x4 matrix     :{}".format(B_4x4.get_matrix()))

# Calculate the size of the tensor as a 2D tuple
print("A_2X2 size       :{}".format(A_2X2.shape))
print("A_4x1 size       :{}".format(A_4x1.shape))
print("A_4x4 size       :{}".format(A_4x4.shape))
print("A_3x1 size       :{}".format(A_3x1.shape))

# Sum/Subtract 2 Matrices of suitable size
print("A_2X2 + B_2x2    :{}".format(A_2X2.add_sub_mat(B_2x2, "add")))
print("A_2X2 - B_2x2    :{}".format(A_2X2.add_sub_mat(B_2x2, "sub")))
print("A_4x4 - B_4x4    :{}".format(A_4x4.add_sub_mat(B_4x4, "sub")))

# Multiply the tensor with tensor of suitable size
print("A_2X2 * B_2x2    :{}".format(A_2X2.multiply_matrices(B_2x2)))

# Sum of a 4x4 with a 4x4 matrix
print("A_4x4 + B_4x4    :{}".format(A_4x4.add_sub_mat(B_4x4, "add")))

# Multiplication of a 4x4 with a 4x4 matrix
print("A_4x4 * B_4x4    :{}".format(A_4x4.multiply_matrices(B_4x4)))

# Multiplication of a 4x4 matrix with a suitable vector
print("A_4x4 * B_4x1    :{}".format(A_4x4.multiply_matrices(B_4x1)))

# Determinant of a 2x2 matrix
print("A_2X2 determinant:{}".format(A_2X2.matrix_determinant()))

# Inverse of a 2x2 matrix
print("A_2X2 inverse    :{}".format(A_2X2.matrix_inverse()))

# A.A(inv) = Identity Matrix
A_2x2_Identity = A_2X2.matrix_inverse().multiply_matrices(A_2X2)
print("A.A(inverse)     :{}".format(A_2x2_Identity))

# Cross Product of 2 suitable tensors
print("A_3X1 x B_3x1    :{}".format(A_3x1.matrix_cross_product(B_3x1)))


Once deleted, variables cannot be recovered. Proceed (y/[n])? y
A_2X2 matrix     :((1, 2), (5, 6))
B_2x2 matrix     :((-1, 3), (-2, 6))
A_4x4 matrix     :((2, 1, 5, 7), (6, -2, 0, 3), (3, 2, -4, 1), (9, 7, 2, 3))
B_4x4 matrix     :((1, 0, 2, 1), (-2, 1, 5, 2), (3, -2, 6, 4), (2, 0, 1, -2))
A_2X2 size       :(2, 2)
A_4x1 size       :(4, 1)
A_4x4 size       :(4, 4)
A_3x1 size       :(3, 1)
A_2X2 + B_2x2    :([0, 5], [3, 12])
A_2X2 - B_2x2    :([2, -1], [7, 0])
A_4x4 - B_4x4    :([1, 1, 3, 6], [8, -3, -5, 1], [0, 4, -10, -3], [7, 7, 1, 5])
A_2X2 * B_2x2    :((-5, 15), (-17, 51))
A_4x4 + B_4x4    :([3, 1, 7, 8], [4, -1, 5, 5], [6, 0, 2, 5], [11, 7, 3, 1])
A_4x4 * B_4x4    :((29, -9, 46, 10), (16, -2, 5, -4), (-11, 10, -7, -11), (7, 3, 68, 25))
A_4x4 * B_4x1    :((63,), (44,), (22,), (37,))
A_2X2 determinant:-4
A_2X2 inverse    :((-1.5, 0.5), (1.25, -0.25))
A.A(inverse)     :((1.0, 0.0), (0.0, 1.0))
A_3X1 x B_3x1    :(-3, 6, -3)


##### Original Code for reference:

In [19]:
%reset
"""Example of 2 x 2 Matrices For Testing"""
a_2x2 = ((1, 2), (5, 6))
b_2x2 = ((-1, 3), (-2, 6))

"""Example of 4 x 4 Matrices For Testing provided by Abhi"""
a_4x4 = ((2, 1, 5, 7), (6, -2, 0, 3), (3, 2, -4, 1), (9, 7, 2, 3))
b_4x4 = ((1, 0, 2, 1), (-2, 1, 5, 2), (3, -2, 6, 4), (2, 0, 1, -2))

"""Example of 3 x 3 Matrix and 3 x 1 column vector For Testing"""
a_3x3 = ((2, 1, 5), (6, -2, 0), (3, 2, -4))
b_3x1 = ((1,), (0,), (2,))



def size_matrix(matrix):
    """Function to check size of a matrix"""
    return [len(matrix), len(matrix[0])]


def matrix_dimensions_check(matrix_a, matrix_b):
    """Function to compare size of matrices"""
    if size_matrix(matrix_a) != size_matrix(matrix_b):
        raise Exception("The matrices are of different dimensions")


def add_sub_mat(a, b, fn_operator):
    """ As per Abhi's advice, the original separate add
    and subtract functions have been combined in to one.
    """
    # Matrix Dimension Check function added in as per Abhi's advice
    matrix_dimensions_check(a, b)

    mat_sum = []
    for r in range(len(a)):
        mat_sum_row = []
        for c in range(len(a[r])):
            temp_value = fn_operator(a[r][c], b[r][c])
            mat_sum_row.append(temp_value)
        mat_sum.append(tuple(mat_sum_row))
    return tuple(mat_sum)


def mat_add(a, b):
    return a + b


def mat_sub(a, b):
    return a - b


def multiply_matrices(a, b):
    """Function to perform multiplication. This methodology was taken from Vipul's
    solution. His method was a lot better than mine in both simplicity and functionality
    """
    if size_matrix(a)[1] != size_matrix(b)[0]:
        raise Exception("The matrices in question cannot be multiplied")
    result = []
    # iterate on row of the first matrix
    for i, mat_list in enumerate(a):
        internal_result = []
        for j in range(len(b[i])):
            # extracts columns from 2nd Matrix for multiplication
            multiplier = list(zip(*b))[j]
            internal_result.append((sum([x*y for x, y in zip(mat_list, multiplier)])))
        result.append(tuple(internal_result))
    return tuple(result)


print("a_2x2 + b_2x2 = {}".format(add_sub_mat(a_2x2, b_2x2, mat_add)))
print("a_2x2 - b_2x2 = {}".format(add_sub_mat(a_2x2, b_2x2, mat_sub)))
print("a_2x2 * b_2x2 = {}".format(multiply_matrices(a_2x2, b_2x2)))

print("a_4x4 + b_4x4 = {}".format(add_sub_mat(a_4x4, b_4x4, mat_add)))
print("a_4x4 - b_4x4 = {}".format(add_sub_mat(a_4x4, b_4x4, mat_sub)))
print("a_4x4 * b_4x4 = {}".format(multiply_matrices(a_4x4, b_4x4)))

print("a_3x3 * b_3x1 = {}".format(multiply_matrices(a_3x3, b_3x1)))

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
a_2x2 + b_2x2 = ((0, 5), (3, 12))
a_2x2 - b_2x2 = ((2, -1), (7, 0))
a_2x2 * b_2x2 = ((-5, 15), (-17, 51))
a_4x4 + b_4x4 = ((3, 1, 7, 8), (4, -1, 5, 5), (6, 0, 2, 5), (11, 7, 3, 1))
a_4x4 - b_4x4 = ((1, 1, 3, 6), (8, -3, -5, 1), (0, 4, -10, -3), (7, 7, 1, 5))
a_4x4 * b_4x4 = ((29, -9, 46, 10), (16, -2, 5, -4), (-11, 10, -7, -11), (7, 3, 68, 25))
a_3x3 * b_3x1 = ((12,), (6,), (-5,))
